### episode 1
函数：$f(x)=x+10sin(5x)+7cos(4x)$

In [1]:
function fitFunCalc(x)
    return x + 10*sin(5*x)+7*cos(4*x)
end

fitFunCalc (generic function with 1 method)

In [30]:
# 将序列转化成对应范围内的数
function series2Number(s, r_max, r_min)
    num = 0
    for i in 1:length(s)
        num = num*2 + s[i]
    end
#     println(num)
#     println(2^length(s))
    return r_min + (r_max-r_min) * num / 2^length(s)
end
function seriesCalc(ge, r_max, r_min)
    sz = size(ge)
    x = zeros(sz[1])
    y = zeros(sz[1])
    for i in 1:sz[1]
        x[i] = series2Number(ge[i, :], r_max, r_min)
        y[i] = fitFunCalc(x[i])
    end
    return x, y
end

seriesCalc (generic function with 1 method)

In [3]:
# select good ones
function selectAndGenerate(ge, r_max, r_min)
    sz = size(ge)
    # 1. 计算值序列
    num, fitValue = seriesCalc(ge, r_max, r_min)
    
    # 2. 归一化
    maxFit, maxIdx = findmax(fitValue)    # 只会返回第一个最大
    minFit, minIdx = findmin(fitValue)
    geBest = ge[maxIdx, :]
    fitBest = fitValue[maxIdx]
    fitValue = (fitValue .- minFit)./(maxFit-minFit)
    
    # 3. 轮盘赌替换,生成新一代
    fv = fitValue./sum(fitValue)         # 使得和为1
    fv = cumsum(fv)                       # 生成序列
    sel = sort(rand(sz[1]))              # 生成选择序列
    nge = zeros(sz)
    newi, fiti = 1, 1
    while newi < sz[1]
        if(sel[newi]<fv[fiti])
            nge[newi, :] = ge[fiti, :]
            newi += 1
        else
            fiti += 1
        end
    end
    return nge, geBest, fitBest
end

selectAndGenerate (generic function with 1 method)

`怎么来理解交叉的影响？`<br />
`解答`：在进化的过程中，高位决定了大范围的分布情况，这些数据会大致判断一下当前所处区域的好坏，在多次淘汰后，一些不太好的区域就被淘汰掉，而这高位的基因也开始趋同

In [4]:
# this function only switch the neighbor
function geneCross!(ge, Pc)
    sz = size(ge)
    for i in 1:2:sz[1]
        # decide if having cross operation
        if rand() < Pc
            select = round.(rand(sz[2]))
            for j in 1:length(select)
                if select[j]>0
                    tmp = ge[i, j]; ge[i, j] = ge[i+1, j]; 
                    ge[i+1, j] = tmp;
                end
            end
        end
    end
end
# test
# 基于概率的不太容易测试

geneCross! (generic function with 1 method)

`变异在搜索过程中的影响`<br />
`解答`：尽量去搜索更多的区域，尽量得到全局的解，但变异的速度显然比不上淘汰的同化，不改变我认为这是低效算法的看法

In [5]:
# this kind of Algorithm dont need to be so accurate
function geneVariation!(ge, Pm)
    for i in 1: size(ge)[1]
        for j in 1:size(ge)[2]
            if rand() < Pm
                ge[i, j] = ge[i, j]>0 ? 0 : 1;
            end
        end
    end
end

geneVariation! (generic function with 1 method)

#### 计算
对这种

In [49]:
using Plots
# 种群数量，编码长度，交叉率，变异率，最大代，上限，下限
NP, L, Pc, Pm, G, Xs,  Xx = 30, 20, 0.6, 0.05, 30, 10.0, 0.0;
ge = Int8.(round.(rand(NP, L)));
cur_x = 0:0.02:10
cur_y = fitFunCalc.(cur_x)
anim = @animate for i in 1:G
    global ge
    plot(cur_x, cur_y)
    x, y = seriesCalc(ge, Xs, Xx)
    plot!(x, y, seriestype=:scatter, title=string(i))
    nge, geBest, fitBest = selectAndGenerate(ge, Xs, Xx)
    geneCross!(nge, Pc)
    geneVariation!(nge, Pm)
    ge = nge
    ge[1, :] = geBest     # 保留上一步的最佳解
end
gif(anim, "anim_fps1.gif", fps = 1)

┌ Info: Saved animation to 
│   fn = /home/kuro/julia/Tutorial_julia/paper1/anim_fps1.gif
└ @ Plots /home/kuro/.julia/packages/Plots/Iuc9S/src/animation.jl:95


Plots.AnimatedGif("/home/kuro/julia/Tutorial_julia/paper1/anim_fps1.gif")

### episode 2 TSP问题
这应该算是一个典型的应用场景了